In [1]:
# Standard library imports
from typing import Optional, List, Tuple
import ast
import os
import random
import re
import time
import shutil
import sys

# Add parent directory to path for imports
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..', '..')))

# Third-party imports
import pandas as pd
from dotenv import load_dotenv
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from typing import Literal
from pydantic import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate

# LangChain imports
from langchain_chroma import Chroma
from langchain_core.documents import Document
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_ollama import ChatOllama
from langchain_openai import ChatOpenAI

# Local imports
from utils_notebook import (
    SemanticCleaner, 
    extract_answer, 
    calcular_metricas_memoria,
    format_choices,
    parse_simple_score,
    make_question
)

cleaner = SemanticCleaner()
# Load environment variables
load_dotenv()

True

In [2]:
df_train = pd.read_csv('../dataset/arc_challenge_train_processed.csv')
df_valid = pd.read_csv('../dataset/arc_challenge_valid_processed.csv')
df_questions = pd.concat([df_train, df_valid], ignore_index=True)

In [3]:
def get_alt_by_question(question_id: str) -> str:  # Mudei para str
    row = df_questions[df_questions['id'] == question_id]
    if not row.empty:
        alternatives_str = row.iloc[0]['choices']
        alternatives = ast.literal_eval(alternatives_str)
        formatted = "\n".join(f"  • {text}" for text in alternatives['text'])
        return formatted
    else:
        return ''

In [4]:
# print(  get_alt_by_question('Mercury_SC_415702'))

In [236]:
gt =  pd.read_csv('scientific_facts_audit_by_gpt_2.csv') 

gt = gt.fillna("N/A")
gt = gt[gt['fact'] != "N/A"].copy()
gt.drop_duplicates(subset=['fact'], keep='first', inplace=True)
#gt = gt[gt['origin'] == 'train']
gt['fact_clean'] = gt['fact'].apply(lambda x: cleaner.clean(x))
print("Quantidade de fatos: ", len(gt))

mapping = {
    'CORRETO': 'CORRECT',
    'ERRADO': 'INCORRECT'
}

# Aplica a substituição na coluna 'verdict'
gt['verdict'] = gt['verdict'].replace(mapping)

Quantidade de fatos:  3993


In [237]:
gt_train = gt[gt['origin'] == 'train'].copy()

In [238]:
our_framework =  pd.read_csv('vecstore_reconstructed_in_csv/our_framework_memory_restored.csv') 

#  0) Number of wrong used facts

### 0.0 Auditor

In [239]:
class FactAuditVerdict(BaseModel):
    verdict: Literal["CORRECT", "INCORRECT"] = Field(
        description="Classify strictly as CORRECT if the fact is scientifically accurate, true, and represents generalizable knowledge. Classify as INCORRECT if it contains scientific errors, hallucinations, is harmful, or represents overly specific/non-generalizable information."
    )
    
# Instanciar LLM e Prompt
auditor = ChatOpenAI(
    model="gpt-5-mini-2025-08-07",
    temperature=0.1,
).with_structured_output(FactAuditVerdict)

audit_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a rigorous scientific fact auditor. Your task is to evaluate whether a scientific fact is:
    
1. CORRECT: The fact is scientifically accurate, true, and represents generalizable scientific knowledge that can be applied broadly.
2. INCORRECT: The fact contains scientific inaccuracies, hallucinations, harmful content, or overly specific/context-dependent information.

Output instructions: 
    - Only output the classification as CORRECT or INCORRECT. Do not provide explanations or justifications, or poncturation.
     
Apply strict scientific standards. When in doubt, prefer marking as INCORRECT."""),
    ("human", "Evaluate this scientific fact:\n\n{fact}")
])

audit_chain = audit_prompt | auditor

import pandas as pd
from tqdm.auto import tqdm

def audit_missing_facts(df, audit_chain, fact_col='retrieved_fact', verdict_col='is_fact_correct'):
    """
    Identifica fatos sem veredito em um DataFrame, executa a auditoria via LLM,
    salva os resultados de forma otimizada e realiza uma verificação de sanidade.
    """
    df_result = df.copy()
    
    # 1. TRATAMENTO DE FATOS VAZIOS: Marca como 'NO_FACT' as linhas onde a extração falhou
    mask_sem_fato = df_result[fact_col].isna() | (df_result[fact_col] == '')
    df_result.loc[mask_sem_fato, verdict_col] = 'NO_FACT'
    
    # 2. IDENTIFICAÇÃO: Filtra o que realmente precisa ser auditado
    mask_nao_auditados = df_result[verdict_col].isna()
    fatos_unicos = df_result.loc[mask_nao_auditados, fact_col].dropna().unique()
    
    if len(fatos_unicos) == 0:
        print("✓ Nenhum fato novo pendente de auditoria.")
        return df_result
        
    print(f"Iniciando auditoria de {len(fatos_unicos)} fatos únicos...")
    audit_results = {}
    
    # 3. AUDITORIA: Loop com a LLM
    pbar = tqdm(fatos_unicos, desc="Auditando")
    for fato in pbar:
        try:
            resultado = audit_chain.invoke({"fact": fato})
            audit_results[fato] = resultado.verdict
            pbar.set_postfix({"Verdict": resultado.verdict})
        except Exception as e:
            audit_results[fato] = None
            pbar.set_postfix({"Erro": "Falha na API"})

    # 4. ATUALIZAÇÃO SEGURA NO DATAFRAME
    resultados_validos = {fato: veredito for fato, veredito in audit_results.items() if veredito is not None}
    
    if resultados_validos:
        # Mapeia os resultados encontrados para as linhas correspondentes
        novos_vereditos = df_result.loc[mask_nao_auditados, fact_col].map(resultados_validos)
        
        # Preenche o DataFrame apenas onde a API retornou sucesso
        df_result.loc[mask_nao_auditados, verdict_col] = novos_vereditos.fillna(df_result.loc[mask_nao_auditados, verdict_col])
        
        # 5. PROVA REAL (Sanity Check)
        for fato_auditado, veredito_esperado in resultados_validos.items():
            linhas_do_fato = df_result[df_result[fact_col] == fato_auditado]
            if not linhas_do_fato.empty:
                veredito_no_df = linhas_do_fato.iloc[0][verdict_col]
                assert veredito_no_df == veredito_esperado, f"ERRO: Fato '{fato_auditado[:30]}...' não foi salvo corretamente!"
                
        print("\n✓ Sanity Check: Todos os resultados válidos da API foram salvos no DataFrame com sucesso.")

    # 6. ESTATÍSTICAS RÁPIDAS
    total = len(df_result)
    corretos = (df_result[verdict_col] == 'CORRECT').sum()
    incorretos = (df_result[verdict_col] == 'INCORRECT').sum()
    sem_fatos = (df_result[verdict_col] == 'NO_FACT').sum()
    pendentes = df_result[verdict_col].isna().sum()
    
    print("-" * 80)
    print(f"RESUMO: Total de Linhas: {total} | ✅ Corretos: {corretos} | ❌ Incorretos: {incorretos} | 📭 Sem Fato: {sem_fatos} | ⚠️ Pendentes: {pendentes}")
    print("-" * 80)
    
    return df_result

## A) Our Solution

In [319]:
our_framework_audicted = pd.read_csv("our_framework_audicted_facts.csv")

of_test = pd.read_csv("hybrid_refinement_cognitive_test_var_07.csv")

In [320]:
def extract_facts(text):
    if not isinstance(text, str):
        return []
    facts = re.findall(r'\*\s*Fact:\s*"{1,2}(.*?)"{1,2}', text)
    return facts

of_test['retrieved_fact'] = of_test['retrieved_context'].apply(extract_facts)
df_expanded = of_test.explode('retrieved_fact').reset_index(drop=True)
# df_expanded.dropna(subset=['retrieved_fact'], inplace=True)

In [322]:
# Verificar se cada fato recuperado é correto ou incorreto segundo o audit
df_expanded['is_fact_correct'] = None
for index, row in df_expanded.iterrows():
    retrieved_fact = df_expanded.at[index, 'retrieved_fact']
    if pd.notna(retrieved_fact) and retrieved_fact in our_framework_audicted['scientific_fact'].values:
        verdict = our_framework_audicted[our_framework_audicted['scientific_fact'] == retrieved_fact]['verdict'].values[0]
        df_expanded.at[index, 'is_fact_correct'] = verdict

In [323]:
df_expanded = audit_missing_facts(df_expanded, audit_chain)

Iniciando auditoria de 2 fatos únicos...


Auditando:   0%|          | 0/2 [00:00<?, ?it/s]


✓ Sanity Check: Todos os resultados válidos da API foram salvos no DataFrame com sucesso.
--------------------------------------------------------------------------------
RESUMO: Total de Linhas: 3511 | ✅ Corretos: 2365 | ❌ Incorretos: 1144 | 📭 Sem Fato: 2 | ⚠️ Pendentes: 0
--------------------------------------------------------------------------------


In [324]:
# Estatísticas sobre os fatos recuperados e sua correção
print("=" * 80)
print("ANÁLISE DOS FATOS RECUPERADOS NO TESTE")
print("=" * 80)

total_fatos = len(df_expanded)
fatos_auditados = df_expanded['is_fact_correct'].notna().sum()
fatos_nao_auditados = df_expanded['is_fact_correct'].isna().sum()

print(f"\nTotal de fatos recuperados (linhas): {total_fatos}")
print(f"Fatos auditados: {fatos_auditados} ({fatos_auditados/total_fatos*100:.2f}%)")
print(f"Fatos não auditados: {fatos_nao_auditados} ({fatos_nao_auditados/total_fatos*100:.2f}%)")

if fatos_auditados > 0:
    print("\nDistribuição dos fatos auditados:")
    print(df_expanded['is_fact_correct'].value_counts(dropna=False))
    
    corretos = (df_expanded['is_fact_correct'] == 'CORRECT').sum()
    incorretos = (df_expanded['is_fact_correct'] == 'INCORRECT').sum()
    
    if corretos + incorretos > 0:
        print(f"\nDos fatos auditados:")
        print(f"  - Corretos: {corretos} ({corretos/(corretos+incorretos)*100:.2f}%)")
        print(f"  - Incorretos: {incorretos} ({incorretos/(corretos+incorretos)*100:.2f}%)")

ANÁLISE DOS FATOS RECUPERADOS NO TESTE

Total de fatos recuperados (linhas): 3511
Fatos auditados: 3511 (100.00%)
Fatos não auditados: 0 (0.00%)

Distribuição dos fatos auditados:
is_fact_correct
CORRECT      2365
INCORRECT    1144
NO_FACT         2
Name: count, dtype: int64

Dos fatos auditados:
  - Corretos: 2365 (67.40%)
  - Incorretos: 1144 (32.60%)


## B) Semantic Base:

In [277]:
semantic = pd.read_csv("semantic_test.csv")

In [278]:
def extract_knowledge(text):
    if not isinstance(text, str):
        return []
    # In semantic_test.csv, the facts are presented as "Knowledge: [fact]"
    facts = re.findall(r'Knowledge:\s*(.*?)(?=\n|$)', text)
    return [f.strip() for f in facts if f.strip()]

semantic['retrieved_fact'] = semantic['retrieved_context'].apply(extract_knowledge)
df_expanded = semantic.explode('retrieved_fact').reset_index(drop=True)
df_expanded.dropna(subset=['retrieved_fact'], inplace=True)

In [247]:
# Verificar se cada fato recuperado é correto ou incorreto segundo o audit
df_expanded['is_fact_correct'] = None
for index, row in df_expanded.iterrows():
    retrieved_fact = df_expanded.at[index, 'retrieved_fact']
    if pd.notna(retrieved_fact) and retrieved_fact in gt['fact_clean'].values:
        verdict = gt[gt['fact_clean'] == retrieved_fact]['verdict'].values[0]
        df_expanded.at[index, 'is_fact_correct'] = verdict

In [248]:
df_expanded = audit_missing_facts(df_expanded, audit_chain)

✓ Nenhum fato novo pendente de auditoria.


In [249]:
# Estatísticas sobre os fatos recuperados e sua correção
print("=" * 80)
print("ANÁLISE DOS FATOS RECUPERADOS NO TESTE")
print("=" * 80)

total_fatos = len(df_expanded)
fatos_auditados = df_expanded['is_fact_correct'].notna().sum()
fatos_nao_auditados = df_expanded['is_fact_correct'].isna().sum()

print(f"\nTotal de fatos recuperados (linhas): {total_fatos}")
print(f"Fatos auditados: {fatos_auditados} ({fatos_auditados/total_fatos*100:.2f}%)")
print(f"Fatos não auditados: {fatos_nao_auditados} ({fatos_nao_auditados/total_fatos*100:.2f}%)")

if fatos_auditados > 0:
    print("\nDistribuição dos fatos auditados:")
    print(df_expanded['is_fact_correct'].value_counts(dropna=False))
    
    corretos = (df_expanded['is_fact_correct'] == 'CORRECT').sum()
    incorretos = (df_expanded['is_fact_correct'] == 'INCORRECT').sum()
    
    if corretos + incorretos > 0:
        print(f"\nDos fatos auditados:")
        print(f"  - Corretos: {corretos} ({corretos/(corretos+incorretos)*100:.2f}%)")
        print(f"  - Incorretos: {incorretos} ({incorretos/(corretos+incorretos)*100:.2f}%)")

ANÁLISE DOS FATOS RECUPERADOS NO TESTE

Total de fatos recuperados (linhas): 3467
Fatos auditados: 3467 (100.00%)
Fatos não auditados: 0 (0.00%)

Distribuição dos fatos auditados:
is_fact_correct
CORRECT      1871
INCORRECT    1596
Name: count, dtype: int64

Dos fatos auditados:
  - Corretos: 1871 (53.97%)
  - Incorretos: 1596 (46.03%)


## C) Semantic Cognitive

In [315]:
semantic_cognitive_test = pd.read_csv('semantic_cognitive_test.csv')

def extract_facts(text):
    if not isinstance(text, str):
        return []
    # In semantic_refinement_cognitive_test.csv, the facts are presented as "Fact: [fact]"
    facts = re.findall(r'Fact:\s*(.*?)(?=\n|$)', text)
    return [f.strip() for f in facts if f.strip()]

semantic_cognitive_test['retrieved_fact'] = semantic_cognitive_test['retrieved_context'].apply(extract_facts)
df_expanded = semantic_cognitive_test.explode('retrieved_fact').reset_index(drop=True)
# df_expanded.dropna(subset=['retrieved_fact'], inplace=True)

In [316]:
# Verificar se cada fato recuperado é correto ou incorreto segundo o audit
df_expanded['is_fact_correct'] = None
for index, row in df_expanded.iterrows():
    retrieved_fact = df_expanded.at[index, 'retrieved_fact']
    if pd.notna(retrieved_fact) and retrieved_fact in gt['fact_clean'].values:
        verdict = gt[gt['fact_clean'] == retrieved_fact]['verdict'].values[0]
        df_expanded.at[index, 'is_fact_correct'] = verdict

In [317]:
df_expanded = audit_missing_facts(df_expanded, audit_chain)

✓ Nenhum fato novo pendente de auditoria.


In [300]:
# Estatísticas sobre os fatos recuperados e sua correção
print("=" * 80)
print("ANÁLISE DOS FATOS RECUPERADOS NO TESTE")
print("=" * 80)

total_fatos = len(df_expanded)
fatos_auditados = df_expanded['is_fact_correct'].notna().sum()
fatos_nao_auditados = df_expanded['is_fact_correct'].isna().sum()

print(f"\nTotal de fatos recuperados (linhas): {total_fatos}")
print(f"Fatos auditados: {fatos_auditados} ({fatos_auditados/total_fatos*100:.2f}%)")
print(f"Fatos não auditados: {fatos_nao_auditados} ({fatos_nao_auditados/total_fatos*100:.2f}%)")

if fatos_auditados > 0:
    print("\nDistribuição dos fatos auditados:")
    print(df_expanded['is_fact_correct'].value_counts(dropna=False))
    
    corretos = (df_expanded['is_fact_correct'] == 'CORRECT').sum()
    incorretos = (df_expanded['is_fact_correct'] == 'INCORRECT').sum()
    
    if corretos + incorretos > 0:
        print(f"\nDos fatos auditados:")
        print(f"  - Corretos: {corretos} ({corretos/(corretos+incorretos)*100:.2f}%)")
        print(f"  - Incorretos: {incorretos} ({incorretos/(corretos+incorretos)*100:.2f}%)")

ANÁLISE DOS FATOS RECUPERADOS NO TESTE

Total de fatos recuperados (linhas): 3514
Fatos auditados: 3514 (100.00%)
Fatos não auditados: 0 (0.00%)

Distribuição dos fatos auditados:
is_fact_correct
CORRECT      2278
INCORRECT    1236
Name: count, dtype: int64

Dos fatos auditados:
  - Corretos: 2278 (64.83%)
  - Incorretos: 1236 (35.17%)


## D) Semantic Cognitive Refined

In [332]:
df = pd.read_csv('semantic_refinement_cognitive_test.csv')

def extract_facts(text):
    if not isinstance(text, str):
        return []
    # Extract everything after "Fact:" until the end of the line
    facts = re.findall(r'Fact:\s*(.*)', text)
    return [f.strip() for f in facts if f.strip()]

df['retrieved_fact'] = df['retrieved_context'].apply(extract_facts)
df_expanded = df.explode('retrieved_fact').reset_index(drop=True)
df_expanded.dropna(subset=['retrieved_fact'], inplace=True) # Aqui retiramos que tinha contexto

In [333]:
# Verificar se cada fato recuperado é correto ou incorreto segundo o audit
df_expanded['is_fact_correct'] = None
for index, row in df_expanded.iterrows():
    retrieved_fact = df_expanded.at[index, 'retrieved_fact']
    if pd.notna(retrieved_fact) and retrieved_fact in gt['fact_clean'].values:
        verdict = gt[gt['fact_clean'] == retrieved_fact]['verdict'].values[0]
        df_expanded.at[index, 'is_fact_correct'] = verdict

In [334]:
df_expanded = audit_missing_facts(df_expanded, audit_chain)

Iniciando auditoria de 141 fatos únicos...


Auditando:   0%|          | 0/141 [00:00<?, ?it/s]


✓ Sanity Check: Todos os resultados válidos da API foram salvos no DataFrame com sucesso.
--------------------------------------------------------------------------------
RESUMO: Total de Linhas: 3509 | ✅ Corretos: 2365 | ❌ Incorretos: 1144 | 📭 Sem Fato: 0 | ⚠️ Pendentes: 0
--------------------------------------------------------------------------------


In [336]:
# Estatísticas finais após auditoria completa
print("\n" + "=" * 80)
print("ESTATÍSTICAS FINAIS - SEMANTIC COGNITIVE REFINED (APÓS AUDITORIA GPT)")
print("=" * 80)

total_fatos = len(df_expanded)
corretos = (df_expanded['is_fact_correct'] == 'CORRECT').sum()
incorretos = (df_expanded['is_fact_correct'] == 'INCORRECT').sum()
nao_auditados = df_expanded['is_fact_correct'].isna().sum()

print(f"\nTotal de fatos recuperados: {total_fatos}")
print(f"\nDistribuição final:")
print(f"  - CORRECT: {corretos} ({corretos/total_fatos*100:.2f}%)")
print(f"  - INCORRECT: {incorretos} ({incorretos/total_fatos*100:.2f}%)")
print(f"  - Não auditados: {nao_auditados} ({nao_auditados/total_fatos*100:.2f}%)")

if corretos + incorretos > 0:
    print(f"\nDos fatos auditados:")
    print(f"  - Corretos: {corretos} ({corretos/(corretos+incorretos)*100:.2f}%)")
    print(f"  - Incorretos: {incorretos} ({incorretos/(corretos+incorretos)*100:.2f}%)")
    
print("\n" + "=" * 80)


ESTATÍSTICAS FINAIS - SEMANTIC COGNITIVE REFINED (APÓS AUDITORIA GPT)

Total de fatos recuperados: 3509

Distribuição final:
  - CORRECT: 2365 (67.40%)
  - INCORRECT: 1144 (32.60%)
  - Não auditados: 0 (0.00%)

Dos fatos auditados:
  - Corretos: 2365 (67.40%)
  - Incorretos: 1144 (32.60%)



## E) Concatenated Cognitive 

In [312]:
semantic_cognitive_reflec_test = pd.read_csv('semantic_cog_reflec_test.csv')

def extract_facts(text):
    if not isinstance(text, str):
        return []
    # Match * Fact: "..." or ""...""
    facts = re.findall(r'\*\s*Fact:\s*"{1,2}(.*?)"{1,2}', text)
    return [f.strip() for f in facts if f.strip()]

semantic_cognitive_reflec_test['retrieved_fact'] = semantic_cognitive_reflec_test['retrieved_context'].apply(extract_facts)
df_expanded = semantic_cognitive_reflec_test.explode('retrieved_fact').reset_index(drop=True)
# df_expanded.dropna(subset=['retrieved_fact'], inplace=True)

# Verificar se cada fato recuperado é correto ou incorreto segundo o audit
df_expanded['is_fact_correct'] = None
for index, row in df_expanded.iterrows():
    retrieved_fact = df_expanded.at[index, 'retrieved_fact']
    if pd.notna(retrieved_fact) and retrieved_fact in gt['fact_clean'].values:
        verdict = gt[gt['fact_clean'] == retrieved_fact]['verdict'].values[0]
        df_expanded.at[index, 'is_fact_correct'] = verdict

In [313]:
# df_expanded = audit_missing_facts(df_expanded, audit_chain)

In [314]:
# Estatísticas sobre os fatos recuperados e sua correção
print("=" * 80)
print("ANÁLISE DOS FATOS RECUPERADOS NO TESTE")
print("=" * 80)

total_fatos = len(df_expanded)
fatos_auditados = df_expanded['is_fact_correct'].notna().sum()
fatos_nao_auditados = df_expanded['is_fact_correct'].isna().sum()

print(f"\nTotal de fatos recuperados (linhas): {total_fatos}")
print(f"Fatos auditados: {fatos_auditados} ({fatos_auditados/total_fatos*100:.2f}%)")
print(f"Fatos não auditados: {fatos_nao_auditados} ({fatos_nao_auditados/total_fatos*100:.2f}%)")

if fatos_auditados > 0:
    print("\nDistribuição dos fatos auditados:")
    print(df_expanded['is_fact_correct'].value_counts(dropna=False))
    
    corretos = (df_expanded['is_fact_correct'] == 'CORRECT').sum()
    incorretos = (df_expanded['is_fact_correct'] == 'INCORRECT').sum()
    
    if corretos + incorretos > 0:
        print(f"\nDos fatos auditados:")
        print(f"  - Corretos: {corretos} ({corretos/(corretos+incorretos)*100:.2f}%)")
        print(f"  - Incorretos: {incorretos} ({incorretos/(corretos+incorretos)*100:.2f}%)")

ANÁLISE DOS FATOS RECUPERADOS NO TESTE

Total de fatos recuperados (linhas): 3477
Fatos auditados: 3463 (99.60%)
Fatos não auditados: 14 (0.40%)

Distribuição dos fatos auditados:
is_fact_correct
CORRECT      2179
INCORRECT    1284
None           14
Name: count, dtype: int64

Dos fatos auditados:
  - Corretos: 2179 (62.92%)
  - Incorretos: 1284 (37.08%)


### F) Concatenated

In [ ]:
concatenate = pd.read_csv('hybrid_test.csv')

def extract_knowledge(text):
    if not isinstance(text, str):
        return []
    # Extracting things after 'Knowledge:'
    facts = re.findall(r'Knowledge:\s*(.*?)(?=\n|$)', text)
    return [f.strip().strip('"\'') for f in facts if f.strip()]

concatenate['retrieved_fact'] = concatenate['retrieved_context'].apply(extract_knowledge)
df_expanded = concatenate.explode('retrieved_fact').reset_index(drop=True)
df_expanded.dropna(subset=['retrieved_fact'], inplace=True)

In [327]:
df_expanded['is_fact_correct'] = None
for index, row in df_expanded.iterrows():
    retrieved_fact = df_expanded.at[index, 'retrieved_fact']
    if pd.notna(retrieved_fact) and retrieved_fact in gt['fact_clean'].values:
        verdict = gt[gt['fact_clean'] == retrieved_fact]['verdict'].values[0]
        df_expanded.at[index, 'is_fact_correct'] = verdict

In [328]:
df_expanded = audit_missing_facts(df_expanded, audit_chain)

✓ Nenhum fato novo pendente de auditoria.


In [329]:
# Estatísticas sobre os fatos recuperados e sua correção
print("=" * 80)
print("ANÁLISE DOS FATOS RECUPERADOS NO TESTE")
print("=" * 80)

total_fatos = len(df_expanded)
fatos_auditados = df_expanded['is_fact_correct'].notna().sum()
fatos_nao_auditados = df_expanded['is_fact_correct'].isna().sum()

print(f"\nTotal de fatos recuperados (linhas): {total_fatos}")
print(f"Fatos auditados: {fatos_auditados} ({fatos_auditados/total_fatos*100:.2f}%)")
print(f"Fatos não auditados: {fatos_nao_auditados} ({fatos_nao_auditados/total_fatos*100:.2f}%)")

if fatos_auditados > 0:
    print("\nDistribuição dos fatos auditados:")
    print(df_expanded['is_fact_correct'].value_counts(dropna=False))
    
    corretos = (df_expanded['is_fact_correct'] == 'CORRECT').sum()
    incorretos = (df_expanded['is_fact_correct'] == 'INCORRECT').sum()
    
    if corretos + incorretos > 0:
        print(f"\nDos fatos auditados:")
        print(f"  - Corretos: {corretos} ({corretos/(corretos+incorretos)*100:.2f}%)")
        print(f"  - Incorretos: {incorretos} ({incorretos/(corretos+incorretos)*100:.2f}%)")

ANÁLISE DOS FATOS RECUPERADOS NO TESTE

Total de fatos recuperados (linhas): 3509
Fatos auditados: 3509 (100.00%)
Fatos não auditados: 0 (0.00%)

Distribuição dos fatos auditados:
is_fact_correct
CORRECT      1899
INCORRECT    1610
Name: count, dtype: int64

Dos fatos auditados:
  - Corretos: 1899 (54.12%)
  - Incorretos: 1610 (45.88%)
